In [98]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import os
import sklearn
from sklearn.metrics import precision_score, recall_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, confusion_matrix, f1_score

from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, ClusterCentroids
from imblearn.ensemble import EasyEnsembleClassifier, BalancedBaggingClassifier
from imblearn.pipeline import Pipeline
from sklearn.datasets import make_classification
import sys

sys.path.append('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/')
from RF_atomver import *


In [99]:
df = pd.read_csv('/Users/jayceepang/msse/capstone/atom2024/atom2024/notebooks/NEK/GP/GP_results/GP_test_results_all_NEK.csv')
df['strategy'] = df['strategy'].astype(str)
df.head(5)

,model,accuracy,precision,recall,specificity,TN,FN,FP,TP,cm,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,feat_type,strategy
0,NEK2_binding_moe_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,moe,scaled
1,NEK2_binding_moe_UNDER_GP,0.738516,0.102564,0.666667,0.741697,201,4,70,8,"[201, 70, 4, 8]",0.704182,0.184161,0.704182,0.177778,NEK2_binding,moe,UNDER
2,NEK2_binding_moe_SMOTE_GP,0.968198,0.800000,0.333333,0.996310,270,8,1,4,"[270, 1, 8, 4]",0.664822,0.504211,0.664822,0.470588,NEK2_binding,moe,SMOTE
3,NEK2_binding_moe_ADASYN_GP,0.961131,0.571429,0.333333,0.988930,268,8,3,4,"[268, 3, 8, 4]",0.661132,0.418102,0.661132,0.421053,NEK2_binding,moe,ADASYN
4,NEK2_binding_mfp_scaled_GP,0.957597,0.000000,0.000000,1.000000,271,12,0,0,"[271, 0, 12, 0]",0.500000,0.000000,0.500000,0.000000,NEK2_binding,mfp,scaled


In [100]:
list1 = list(df.columns)
list2 = ['recall', 'precision']
list3 = [m for m in list1 if m not in list2]
# list3

In [101]:
def rank(df, compare_type, metrics): 
    # metrics = ['recall', 'precision']
    # compare_type = 'NEK'

    if not isinstance(compare_type, list):
        compare_type = [compare_type]
    copydf = df.copy()
    # temp_groupby = copydf.groupby(compare_type, group_keys=False)[metrics].agg('max')
    # print(temp_groupby)
    grouped = df.groupby(compare_type, group_keys=False).apply(lambda x:x.sort_values(by=metrics, ascending=[False]*len(metrics))) 
        # can do any lsit of metrics 
    cols = list(df.columns) 
    # keep_cols = ['model','model_type','NEK', 'feat_type','cm']
    # keep_cols = ['model','cm']
    # keep_cols.append(compare_type)
    # keep_cols.extend(compare_type)
    # keep_cols.append(metrics) 
    # keep_cols.extend(metrics)
    keep_cols = ['model', 'cm'] + compare_type + metrics
 
    # other_cols = [col for col in cols if col not in keep_cols]
    # print(cols)
    # print(other_cols)
    ranked_df = grouped.reset_index(drop=True)[keep_cols]
    # ranked_df = grouped.reset_index(drop=True).drop(columns=other_cols)

    ranked_df
    return ranked_df

In [102]:
neks = ['NEK2_binding', 'NEK2_inhibition', 'NEK3_binding', 'NEK5_binding', 'NEK9_binding', 'NEK9_inhibition']
for n in neks: 
    nek_df = df[df['NEK'] == n]
    compare_types = ['NEK','strategy'] 
    print(n)
    # metrics=['recall', 'precision','ROC-AUC', 'f1']
    all_metrics = ['accuracy',	'precision',	'recall',	'specificity',	'ROC-AUC',	'MCC',	'Balanced Accuracy' ,'f1'] 
    metrics = ['recall','accuracy', 'precision', 'ROC-AUC', 'MCC', 'f1']
    ranked = rank(nek_df, compare_types,metrics)
    # print(ranked)
    display(ranked)
    print('\n\n')

NEK2_binding


,model,cm,NEK,strategy,recall,accuracy,precision,ROC-AUC,MCC,f1
0,NEK2_binding_moe_scaled_GP,"[271, 0, 12, 0]",NEK2_binding,scaled,0.000000,0.957597,0.000000,0.500000,0.000000,0.000000
1,NEK2_binding_moe_UNDER_GP,"[201, 70, 4, 8]",NEK2_binding,UNDER,0.666667,0.738516,0.102564,0.704182,0.184161,0.177778
2,NEK2_binding_moe_SMOTE_GP,"[270, 1, 8, 4]",NEK2_binding,SMOTE,0.333333,0.968198,0.800000,0.664822,0.504211,0.470588
3,NEK2_binding_moe_ADASYN_GP,"[268, 3, 8, 4]",NEK2_binding,ADASYN,0.333333,0.961131,0.571429,0.661132,0.418102,0.421053
4,NEK2_binding_mfp_scaled_GP,"[271, 0, 12, 0]",NEK2_binding,scaled,0.000000,0.957597,0.000000,0.500000,0.000000,0.000000
5,NEK2_binding_mfp_UNDER_GP,"[270, 1, 12, 0]",NEK2_binding,UNDER,0.000000,0.954064,0.000000,0.498155,-0.012531,0.000000
6,NEK2_binding_mfp_SMOTE_GP,"[271, 0, 10, 2]",NEK2_binding,SMOTE,0.166667,0.964664,1.000000,0.583333,0.400918,0.285714
7,NEK2_binding_mfp_ADASYN_GP,"[271, 0, 10, 2]",NEK2_binding,ADASYN,0.166667,0.964664,1.000000,0.583333,0.400918,0.285714





NEK2_inhibition


,model,cm,NEK,strategy,recall,accuracy,precision,ROC-AUC,MCC,f1
0,NEK2_inhibition_mfp_ADASYN_GP,"[379, 2, 12, 16]",NEK2_inhibition,ADASYN,0.571429,0.965770,0.888889,0.783090,0.697061,0.695652
1,NEK2_inhibition_moe_ADASYN_GP,"[374, 7, 13, 15]",NEK2_inhibition,ADASYN,0.535714,0.951100,0.681818,0.758671,0.579098,0.600000
2,NEK2_inhibition_mfp_SMOTE_GP,"[379, 2, 11, 17]",NEK2_inhibition,SMOTE,0.607143,0.968215,0.894737,0.800947,0.722191,0.723404
3,NEK2_inhibition_moe_SMOTE_GP,"[374, 7, 14, 14]",NEK2_inhibition,SMOTE,0.500000,0.948655,0.666667,0.740814,0.551096,0.571429
4,NEK2_inhibition_mfp_UNDER_GP,"[16, 365, 1, 27]",NEK2_inhibition,UNDER,0.964286,0.105134,0.068878,0.503140,0.007946,0.128571
5,NEK2_inhibition_moe_UNDER_GP,"[332, 49, 6, 22]",NEK2_inhibition,UNDER,0.785714,0.865526,0.309859,0.828553,0.438116,0.444444
6,NEK2_inhibition_mfp_scaled_GP,"[381, 0, 23, 5]",NEK2_inhibition,scaled,0.178571,0.943765,1.000000,0.589286,0.410372,0.303030
7,NEK2_inhibition_moe_scaled_GP,"[381, 0, 28, 0]",NEK2_inhibition,scaled,0.000000,0.931540,0.000000,0.500000,0.000000,0.000000





NEK3_binding


...

,model,cm,NEK,strategy,recall,accuracy,precision,ROC-AUC,MCC,f1
0,NEK5_binding_moe_ADASYN_GP,"[220, 8, 7, 13]",NEK5_binding,ADASYN,0.65,0.939516,0.619048,0.807456,0.601413,0.634146
1,NEK5_binding_mfp_ADASYN_GP,"[226, 2, 10, 10]",NEK5_binding,ADASYN,0.50,0.951613,0.833333,0.745614,0.623331,0.625000
2,NEK5_binding_mfp_SMOTE_GP,"[226, 2, 9, 11]",NEK5_binding,SMOTE,0.55,0.955645,0.846154,0.770614,0.661237,0.666667
3,NEK5_binding_moe_SMOTE_GP,"[219, 9, 9, 11]",NEK5_binding,SMOTE,0.55,0.927419,0.550000,0.755263,0.510526,0.550000
4,NEK5_binding_mfp_UNDER_GP,"[125, 103, 9, 11]",NEK5_binding,UNDER,0.55,0.548387,0.096491,0.549123,0.053677,0.164179
5,NEK5_binding_moe_UNDER_GP,"[228, 0, 20, 0]",NEK5_binding,UNDER,0.00,0.919355,0.000000,0.500000,0.000000,0.000000
6,NEK5_binding_mfp_scaled_GP,"[228, 0, 19, 1]",NEK5_binding,scaled,0.05,0.923387,1.000000,0.525000,0.214834,0.095238
7,NEK5_binding_moe_scaled_GP,"[228, 0, 20, 0]",NEK5_binding,scaled,0.00,0.919355,0.000000,0.500000,0.000000,0.000000





NEK9_binding


,model,cm,NEK,strategy,recall,accuracy,precision,ROC-AUC,MCC,f1
0,NEK9_binding_mfp_ADASYN_GP,"[268, 2, 12, 1]",NEK9_binding,ADASYN,0.076923,0.950530,0.333333,0.534758,0.142101,0.125000
1,NEK9_binding_moe_ADASYN_GP,"[266, 4, 12, 1]",NEK9_binding,ADASYN,0.076923,0.943463,0.200000,0.531054,0.098695,0.111111
2,NEK9_binding_mfp_SMOTE_GP,"[268, 2, 12, 1]",NEK9_binding,SMOTE,0.076923,0.950530,0.333333,0.534758,0.142101,0.125000
3,NEK9_binding_moe_SMOTE_GP,"[266, 4, 12, 1]",NEK9_binding,SMOTE,0.076923,0.943463,0.200000,0.531054,0.098695,0.111111
4,NEK9_binding_moe_UNDER_GP,"[172, 98, 7, 6]",NEK9_binding,UNDER,0.461538,0.628975,0.057692,0.549288,0.042804,0.102564
5,NEK9_binding_mfp_UNDER_GP,"[270, 0, 13, 0]",NEK9_binding,UNDER,0.000000,0.954064,0.000000,0.500000,0.000000,0.000000
6,NEK9_binding_moe_scaled_GP,"[270, 0, 13, 0]",NEK9_binding,scaled,0.000000,0.954064,0.000000,0.500000,0.000000,0.000000
7,NEK9_binding_mfp_scaled_GP,"[270, 0, 13, 0]",NEK9_binding,scaled,0.000000,0.954064,0.000000,0.500000,0.000000,0.000000





NEK9_inhibition


,model,cm,NEK,strategy,recall,accuracy,precision,ROC-AUC,MCC,f1
0,NEK9_inhibition_moe_ADASYN_GP,"[70, 1, 3, 6]",NEK9_inhibition,ADASYN,0.666667,0.9500,0.857143,0.826291,0.729752,0.750000
1,NEK9_inhibition_mfp_ADASYN_GP,"[70, 1, 3, 6]",NEK9_inhibition,ADASYN,0.666667,0.9500,0.857143,0.826291,0.729752,0.750000
2,NEK9_inhibition_moe_SMOTE_GP,"[70, 1, 3, 6]",NEK9_inhibition,SMOTE,0.666667,0.9500,0.857143,0.826291,0.729752,0.750000
3,NEK9_inhibition_mfp_SMOTE_GP,"[70, 1, 3, 6]",NEK9_inhibition,SMOTE,0.666667,0.9500,0.857143,0.826291,0.729752,0.750000
4,NEK9_inhibition_moe_UNDER_GP,"[0, 71, 0, 9]",NEK9_inhibition,UNDER,1.000000,0.1125,0.112500,0.500000,0.000000,0.202247
5,NEK9_inhibition_mfp_UNDER_GP,"[71, 0, 9, 0]",NEK9_inhibition,UNDER,0.000000,0.8875,0.000000,0.500000,0.000000,0.000000
6,NEK9_inhibition_mfp_scaled_GP,"[70, 1, 4, 5]",NEK9_inhibition,scaled,0.555556,0.9375,0.833333,0.770736,0.649582,0.666667
7,NEK9_inhibition_moe_scaled_GP,"[70, 1, 9, 0]",NEK9_inhibition,scaled,0.000000,0.8750,0.000000,0.492958,-0.040057,0.000000


In [12]:
nek2_bind = df[df['NEK'] == 'NEK2_binding']
# print(nek2_bind['strategy'].unique())
# grouped = nek2_bind.groupby('strategy')
# for name, group in grouped:
    # print(f'group: {name}')
    # print(group)
# metrics=['recall']
nek2_ranked = rank(nek2_bind, 'strategy', ['recall', 'f1'])
nek2_ranked

,model,cm,strategy,recall,f1
0,NEK2_binding_moe_scaled_GP,"[271, 0, 12, 0]",scaled,0.000000,0.000000
1,NEK2_binding_moe_UNDER_GP,"[201, 70, 4, 8]",UNDER,0.666667,0.177778
2,NEK2_binding_moe_SMOTE_GP,"[270, 1, 8, 4]",SMOTE,0.333333,0.470588
3,NEK2_binding_moe_ADASYN_GP,"[268, 3, 8, 4]",ADASYN,0.333333,0.421053
4,NEK2_binding_mfp_scaled_GP,"[271, 0, 12, 0]",scaled,0.000000,0.000000
5,NEK2_binding_mfp_UNDER_GP,"[270, 1, 12, 0]",UNDER,0.000000,0.000000
6,NEK2_binding_mfp_SMOTE_GP,"[271, 0, 10, 2]",SMOTE,0.166667,0.285714
7,NEK2_binding_mfp_ADASYN_GP,"[271, 0, 10, 2]",ADASYN,0.166667,0.285714


In [103]:
neks = ['NEK2_binding', 'NEK2_inhibition','NEK3_binding', 'NEK5_binding', 'NEK9_binding','NEK9_inhibition']
allrecallvcounts = []
for n, nek in enumerate(neks): 
    nek_df = df[df['NEK'] ==nek]
    print(nek)
    
    # metrics = ['accuracy',	'precision',	'recall',	'specificity',	'ROC-AUC',	'MCC',	'Balanced Accuracy' ,'f1'] 
    metrics=['recall']
    df_result = pd.DataFrame(columns=metrics)
    results=[]
    
    # for nek in neks:
    #     nek_df = df[df['NEK'] == nek]
        
        
    for strategy in nek_df['strategy'].unique():
        strat_df = nek_df[nek_df['strategy'] == strategy]
        model = nek_df['model'] 
        # display(strat_df)
        best_feat_types = {}

        for met in metrics:
            best_feat_type = strat_df.groupby('feat_type')[met].max().idxmax()
            best_feat_types[met] = best_feat_type

        best_feat_types['NEK'] = nek
        best_feat_types['strategy'] = strategy
        best_feat_types['model'] = 'GP'
        results.append(best_feat_types)

    final_results = pd.DataFrame(results)
    display(final_results)
    recall_value_counts = final_results['recall'].value_counts()
    print(recall_value_counts)
    print()
    allrecallvcounts.append(recall_value_counts)
        
        

NEK2_binding


,recall,NEK,strategy,model
0,mfp,NEK2_binding,scaled,GP
1,moe,NEK2_binding,UNDER,GP
2,moe,NEK2_binding,SMOTE,GP
3,moe,NEK2_binding,ADASYN,GP


recall
moe    3
mfp    1
Name: count, dtype: int64

NEK2_inhibition


,recall,NEK,strategy,model
0,mfp,NEK2_inhibition,scaled,GP
1,mfp,NEK2_inhibition,UNDER,GP
2,mfp,NEK2_inhibition,SMOTE,GP
3,mfp,NEK2_inhibition,ADASYN,GP


recall
mfp    4
Name: count, dtype: int64

NEK3_binding


...

recall
mfp    3
moe    1
Name: count, dtype: int64

NEK9_binding


,recall,NEK,strategy,model
0,mfp,NEK9_binding,scaled,GP
1,moe,NEK9_binding,UNDER,GP
2,mfp,NEK9_binding,SMOTE,GP
3,mfp,NEK9_binding,ADASYN,GP


recall
mfp    3
moe    1
Name: count, dtype: int64

NEK9_inhibition


,recall,NEK,strategy,model
0,mfp,NEK9_inhibition,scaled,GP
1,moe,NEK9_inhibition,UNDER,GP
2,mfp,NEK9_inhibition,SMOTE,GP
3,mfp,NEK9_inhibition,ADASYN,GP


recall
mfp    3
moe    1
Name: count, dtype: int64



In [43]:
allrecallvcounts

[recall
 moe    3
 mfp    1
 Name: count, dtype: int64,
 recall
 mfp    4
 Name: count, dtype: int64,
 recall
 mfp    4
 Name: count, dtype: int64,
 recall
 mfp    3
 moe    1
 Name: count, dtype: int64,
 recall
 mfp    3
 moe    1
 Name: count, dtype: int64,
 recall
 mfp    3
 moe    1
 Name: count, dtype: int64]

In [49]:
neks = ['NEK2_binding', 'NEK2_inhibition','NEK3_binding', 'NEK5_binding', 'NEK9_binding','NEK9_inhibition']
recallbystrat=[] 
for n, nek in enumerate(neks): 
    nek_df = df[df['NEK'] ==nek]
    print(nek)
    
    metrics = ['accuracy',	'precision',	'recall',	'specificity',	'ROC-AUC',	'MCC',	'Balanced Accuracy' ,'f1'] 
    df_result = pd.DataFrame(columns=metrics)
    results=[]
    for feat in nek_df['feat_type'].unique():
        feat_df = nek_df[nek_df['feat_type'] == feat]
        model = nek_df['model'] 
    
        # display(strat_df)
        best_strat_types = {}
    
        for met in metrics:
            best_strat_type = feat_df.groupby('strategy')[met].max().idxmax()
            best_strat_types[met] = best_strat_type
    
        best_strat_types['NEK'] = nek
        best_strat_types['feat_type'] =feat
        best_strat_types['model'] = 'GP'
    
        results.append(best_strat_types)

    final_results = pd.DataFrame(results)
    display(final_results)
    recall_value_counts = final_results['recall'].value_counts()
    print(recall_value_counts)
    print()
    recallbystrat.append(recall_value_counts)
        
    

NEK2_binding


,accuracy,precision,recall,specificity,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,feat_type,model
0,SMOTE,SMOTE,UNDER,scaled,UNDER,SMOTE,UNDER,SMOTE,NEK2_binding,moe,GP
1,ADASYN,ADASYN,ADASYN,ADASYN,ADASYN,ADASYN,ADASYN,ADASYN,NEK2_binding,mfp,GP


recall
UNDER     1
ADASYN    1
Name: count, dtype: int64

NEK2_inhibition


,accuracy,precision,recall,specificity,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,feat_type,model
0,ADASYN,ADASYN,UNDER,scaled,UNDER,ADASYN,UNDER,ADASYN,NEK2_inhibition,moe,GP
1,SMOTE,scaled,UNDER,scaled,SMOTE,SMOTE,SMOTE,SMOTE,NEK2_inhibition,mfp,GP


recall
UNDER    2
Name: count, dtype: int64

NEK3_binding


...

recall
ADASYN    1
SMOTE     1
Name: count, dtype: int64

NEK9_binding


,accuracy,precision,recall,specificity,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,feat_type,model
0,scaled,ADASYN,UNDER,scaled,UNDER,ADASYN,UNDER,ADASYN,NEK9_binding,moe,GP
1,UNDER,ADASYN,ADASYN,UNDER,ADASYN,ADASYN,ADASYN,ADASYN,NEK9_binding,mfp,GP


recall
UNDER     1
ADASYN    1
Name: count, dtype: int64

NEK9_inhibition


,accuracy,precision,recall,specificity,ROC-AUC,MCC,Balanced Accuracy,f1,NEK,feat_type,model
0,ADASYN,ADASYN,UNDER,ADASYN,ADASYN,ADASYN,ADASYN,ADASYN,NEK9_inhibition,moe,GP
1,ADASYN,ADASYN,ADASYN,UNDER,ADASYN,ADASYN,ADASYN,ADASYN,NEK9_inhibition,mfp,GP


recall
UNDER     1
ADASYN    1
Name: count, dtype: int64



In [50]:
recallbystrat

[recall
 UNDER     1
 ADASYN    1
 Name: count, dtype: int64,
 recall
 UNDER    2
 Name: count, dtype: int64,
 recall
 ADASYN    1
 UNDER     1
 Name: count, dtype: int64,
 recall
 ADASYN    1
 SMOTE     1
 Name: count, dtype: int64,
 recall
 UNDER     1
 ADASYN    1
 Name: count, dtype: int64,
 recall
 UNDER     1
 ADASYN    1
 Name: count, dtype: int64]

In [95]:
neks = ['NEK2_binding', 'NEK2_inhibition','NEK3_binding', 'NEK5_binding', 'NEK9_binding','NEK9_inhibition']
series_feat_strat = [] 
for n, nek in enumerate(neks): 
    nek_df = df[df['NEK'] ==nek]
    print(nek)

    groupedby_stratfeat = nek_df.groupby(['strategy', 'feat_type'])['recall'].max().idxmax()
    # display(groupedby_strat)
    groupedby_strat = nek_df.groupby(['strategy'])['recall'].max().idxmax()
    # display(groupedby_strat)
    groupedby_feat = nek_df.groupby(['feat_type'])['recall'].max().idxmax()
    # display(groupedby_feat)
    for strat in nek_df['strategy'].unique():
        strat_df = nek_df[nek_df['strategy'] == strat]
        model = nek_df['model'] 
        NEK = nek_df['NEK']
        best_feat = strat_df.groupby([strat])['recall'].max().idxmax()
        display(strat_df)
        selected_columns = strat_df[['model', 'NEK', 'strategy', 'feat_type', 'recall']]
        
        series_feat_strat.append(selected_columns)

        # series_feat_strat.append(strat_df['model', 'NEK','strategy', 'feat_type', 'recall'])
    
    #     # display(strat_df)
    #     best_strat_types = {}
    
    #     for met in metrics:
    #         best_strat_type = feat_df.groupby('strategy')[met].max().idxmax()
    #         best_strat_types[met] = best_strat_type
    
    #     best_strat_types['NEK'] = nek
    #     best_strat_types['feat_type'] =feat
    #     best_strat_types['model'] = 'GP'
    
    #     results.append(best_strat_types)

    # final_results = pd.DataFrame(results)
    # display(final_results)
    # recall_value_counts = final_results['recall'].value_counts()
    # print(recall_value_counts)
    # print()
    # recallbystrat.append(recall_value_counts)
        
    

NEK2_binding


ValueError: attempt to get argmax of an empty sequence

In [84]:
series_feat_strat
all_df= pd.DataFrame()
for i, df in enumerate(series_feat_strat): 
    all_df = pd.concat([all_df, df])
for n in all_df['NEK'].unique():
    nek=n
    strategy = 

NEK2_binding
NEK2_inhibition
NEK3_binding
NEK5_binding
NEK9_binding
NEK9_inhibition


In [85]:
all_df

,model,NEK,strategy,feat_type,recall
0,NEK2_binding_moe_scaled_GP,NEK2_binding,scaled,moe,0.000000
4,NEK2_binding_mfp_scaled_GP,NEK2_binding,scaled,mfp,0.000000
1,NEK2_binding_moe_UNDER_GP,NEK2_binding,UNDER,moe,0.666667
5,NEK2_binding_mfp_UNDER_GP,NEK2_binding,UNDER,mfp,0.000000
2,NEK2_binding_moe_SMOTE_GP,NEK2_binding,SMOTE,moe,0.333333
6,NEK2_binding_mfp_SMOTE_GP,NEK2_binding,SMOTE,mfp,0.166667
3,NEK2_binding_moe_ADASYN_GP,NEK2_binding,ADASYN,moe,0.333333
7,NEK2_binding_mfp_ADASYN_GP,NEK2_binding,ADASYN,mfp,0.166667
8,NEK2_inhibition_moe_scaled_GP,NEK2_inhibition,scaled,moe,0.000000
12,NEK2_inhibition_mfp_scaled_GP,NEK2_inhibition,scaled,mfp,0.178571


In [ ]:
neks = ['NEK2_binding', 'NEK2_inhibition','NEK3_binding', 'NEK5_binding', 'NEK9_binding','NEK9_inhibition']
allrecallvcounts = []
for n, nek in enumerate(neks): 
    nek_df = df[df['NEK'] ==nek]
    print(nek)
    
    # metrics = ['accuracy',	'precision',	'recall',	'specificity',	'ROC-AUC',	'MCC',	'Balanced Accuracy' ,'f1'] 
    metrics=['recall']
    df_result = pd.DataFrame(columns=metrics)
    results=[]
    for strategy in nek_df['strategy'].unique():
        strat_df = nek_df[nek_df['strategy'] == strategy]
        model = nek_df['model'] 
        # display(strat_df)
        best_feat_types = {}

        for met in metrics:
            best_feat_type = strat_df.groupby('feat_type')[met].max().idxmax()
            best_feat_types[met] = best_feat_type

        best_feat_types['NEK'] = nek
        best_feat_types['strategy'] = strategy
        best_feat_types['model'] = 'GP'
        results.append(best_feat_types)

    final_results = pd.DataFrame(results)
    display(final_results)
    recall_value_counts = final_results['recall'].value_counts()
    print(recall_value_counts)
    print()
    allrecallvcounts.append(recall_value_counts)
        
        